In [1]:
import numpy as np
from numpy import linalg
import cvxopt
import cvxopt.solvers

In [2]:
def linear_kernel(x1, x2):
    return np.dot(x1, x2)

In [3]:
def polynomial_kernel(x, y, p=3):
    return (1 + np.dot(x, y)) ** p

In [4]:
def gaussian_kernel(x, y, sigma=5.0):
    return np.exp((-linalg.norm(x - y) **2) / (2 * (sigma ** 2)))

In [9]:
class SVM(object):
    def __init__(self, kernel=linear_kernel, C = None):
        self.kernel = kernel
        self.C = C
        if self.C is not None: self.C = float(self.C)
            
    def fit(self, X, y):
        n_samples, n_features = X.shape
        
        #Gram matrix
        K = np.zeros((n_samples, n_samples))
        for i in range(n_samples):
            for j in range(n_samples):
                K[i,j] = self.kernel(X[i], X[j])
                
        P = cvxopt.matrix(np.outer(y,y) * K)
        q = cvxopt.matrix(np.ones(n_samples) * -1)
        A = cvxopt.matrix(y, (1, n_samples))
        b = cvxopt.matrix(0.0)
        
        if self.C is None:
            G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
            h = cvxopt.matrix(np.zeros(n_samples))
        else:
            tmp1 = np.diag(np.ones(n_samples) * -1)
            tmp2 = np.identity(n_samples)
            G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
            tmp1 = np.zeros(n_samples)
            tmp2 = np.ones(n_samples) * self.C
            h = cvxopt.matrix(np.hstack((tmp1, tmp2)))
            
        #solve QP problem (Quadratic Programming)
        solution = cvxopt.solvers.qp(P, q, G, h, A, b)
        
        #Lagrange multipliers
        a = np.ravel(solution['x'])
        
        #Support vector have non zero lagrange multipliers
        sv =  a > 1e-5
        ind = np.arange(len(a))[sv]
        self.a = a[sv]
        self.sv = X[sv]
        self.sv_y = y[sv]
        
        print("%d support vectors out of %d points" % (len(self.a), n_samples))
        
        # Intercept
        self.b = 0
        for i in range(len(self.a)):
            self.b += self.sv_y[i]
            self.b -= np.sum(self.a * self.sv_y * K[ind[i], sv])
        self.b /= len(self.a)
        
        #Weight vector
        if self.kernel == linear_kernel:
            self.w = np.zeros(n_features)
            for n in range(len(self.a)):
                self.w += self.a[n] * self.sv_y[n] * self.sv[n]
        else:
            self.w = None
            
    def project(self, X):
        if self.w is not None:
            return np.dot(X, self.w) + self.b
        else:
            y_predict = np. zeros(len(X))
            for i in range(len(X)):
                s = 0
                for a, sv_y, sv in zip(self.a, self.sv_y, self.sv):
                    s += a * sv_y * self.kernel(X[i], sv)
                
                y_predict[i] = s
            return y_predict + self.b
        
    def predict(self, X):
        return np.sign(self.project(X))


In [14]:
if __name__ == '__main__':
    import pylab as pl
    
    def gen_lin_separable_data():
        #generate training data in the 2D case
        mean1 = np.array([0,2])
        mean2 = np.array([2,0])
        cov = np.array([[0.8, 0.6], [0.6, 0.8]])
        X1 = np.random.multivariate_normal(mean1, cov, 100)
        y1 = np.ones(len(X1))
        X2 = np.random.multivariate_normal(mean2, cov, 100)
        y2 = np.ones(len(X2)) * -1
        return X1, y1, X2, y2
    
    def gen_non_lin_separable_data():
        mean1 = [-1, 2]
        mean2 = [1, -1]
        mean3 = [4, -4]
        mean4 = [-4, 4]
        cov = [[1.0, 0.8], [0.8, 1.0]]
        X1 = np.random.multivariate_normal(mean1, cov, 50)
        X1 = np.vstack((X1, np.random.multivariate_normal(mean3, cov, 50)))
        y1 = np.ones(len(X1))
        X2 = np.random.multivariate_normal(mean2, cov, 50)
        X2 = np.vstack((X2, np.random.multivariate_normal(mean4, cov, 50)))
        y2 = np.ones(len(X2)) * -1
        return X1, y1, X2, y2
    
    def gen_lin_separable_overlap_data():
        #generate training data in 2-D case
        mean1 = np.array([0,2])
        mean2 = np.array([2,0])
        cov = np.array([[1.5, 1.0], [1.0, 1.5]])
        X1 = np.random.multivariate_normal(mean1, cov, 100)
        y1 = np.ones(len(X1))
        X2 = np.random.multivariate_normal(mean2, cov, 100)
        y2 = np.ones(len(X2)) * -1
        return X1, y1, X2, y2
        
    def split_train(X1, y1, X2, y2):
        X1_train = X1[:90]
        y1_train = y1[:90]
        X2_train = X2[:90]
        y2_train = y2[:90]
        X_train = np.vstack((X1_train, X2_train))
        y_train = np.hstack((y1_train, y2_train))
        return X_train, y_train
        
    def split_test(X1, y1, X2, y2):
        X1_test = X1[90:]
        y1_test = y1[90:]
        X2_test = X2[90:]
        y2_test = y2[90:]
        X_test = np.vstack((X1_test, X2_test))
        y_test = np.hstack((y1_test, y2_test))
        return X_test, y_test
    
    def plot_margin(X1_train, X2_train, clf):
        def f(x, w, b, c=0):
            # given x, return y such that [x,y] in on the line
            # w.x + b = c
            return (-w[0] * x - b + c) / w[1]
        
        pl.plot(X1_train[:, 0], X1_train[:, 1], "ro" )
        pl.plot(X2_train[:, 0], X2_train[:, 1], "bo" )
        pl.scatter(clf.sv[:, 0], clf.sv[:, 1], s=100, c="g")
        
        # w.x + b = 1
        a0 = -4; a1 = f(a0, clf.w, clf.b, 1)
        b0 = 4; b1 = f(b0, clf.w, clf.b, 1)
        pl.plot([a0, b0], [a1, b1], "k--")

        # w.x + b = 0
        a0 = -4; a1 = f(a0, clf.w, clf.b)
        b0 = 4; b1 = f(b0, clf.w, clf.b)
        pl.plot([a0, b0], [a1, b1], "k")

        # w.x + b = -1
        a0 = -4; a1 = f(a0, clf.w, clf.b, -1)
        b0 = 4; b1 = f(b0, clf.w, clf.b, -1)
        pl.plot([a0, b0], [a1, b1], "k--")
        
        pl.axis("tight")
        pl.show()
        
    def plot_contour(X1_train, X2_train, clf):
        pl.plot(X1_train[:, 0], X1_train[:, 1], "ro" )
        pl.plot(X2_train[:, 0], X2_train[:, 1], "bo" )
        pl.scatter(clf.sv[:, 0], clf.sv[:, 1], s=100, c="g")
        
        X1, X2 = np.meshgrid(np.linspace(-6, 6, 50), np.linspace(-6, 6, 50))
        X = np.array([[x1, x2] for x1, x2 in zip(np.ravel(X1), np.ravel(X2))])
        Z = clf.project(X).reshape(X1.shape)
        pl.contour(X1, X2, Z, [0.0], colors='k', linewidths=1, origin='lower')
        pl.contour(X1, X2, Z + 1, [0.0], colors='grey', linewidths=1, origin='lower')
        pl.contour(X1, X2, Z - 1, [0.0], colors='grey', linewidths=1, origin='lower')
        
        pl.axis("tight")
        pl.show()
        
    def test_linear():
        X1, y1, X2, y2 = gen_lin_separable_data()
        X_train, y_train = split_train(X1, y1, X2, y2)
        X_test, y_test = split_test(X1, y1, X2, y2)
        
        clf = SVM()
        clf.fit(X_train, y_train)
        
        y_predict = clf.predict(X_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        
        plot_margin(X_train[y_train == 1], X_train[y_train == -1], clf)
        
    def test_non_linear():
        X1, y1, X2, y2 = gen_non_lin_separable_data()
        X_train, y_train = split_train(X1, y1, X2, y2)
        X_test, y_test = split_test(X1, y1, X2, y2)
        
        clf = SVM(polynomial_kernel)
        clf.fit(X_train, y_train)
        
        y_predict = clf.predict(X_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        
        plot_contour(X_train[y_train == 1], X_train[y_train == -1], clf)

    def test_soft():
        X1, y1, X2, y2 = gen_lin_separable_overlap_data()
        X_train, y_train = split_train(X1, y1, X2, y2)
        X_test, y_test = split_test(X1, y1, X2, y2)
        
        clf = SVM(C=1000.1)
        clf.fit(X_train, y_train)
        
        y_predict = clf.predict(X_test)
        correct = np.sum(y_predict == y_test)
        print("%d out of %d predictions correct" % (correct, len(y_predict)))
        
        plot_contour(X_train[y_train == 1], X_train[y_train == -1], clf)

    #test_linear()
    #test_non_linear()
    test_soft()
        

     pcost       dcost       gap    pres   dres
 0:  1.3275e+05 -7.5499e+07  2e+08  5e-01  5e-12
 1:  3.3321e+05 -1.3540e+07  2e+07  5e-02  1e-11
 2:  3.3219e+05 -4.3932e+06  7e+06  1e-02  5e-12
 3:  1.8638e+05 -1.0963e+06  1e+06  2e-03  3e-12
 4:  2.8742e+04 -1.7710e+05  2e+05  1e-04  4e-12
 5: -1.9576e+03 -2.0330e+04  2e+04  8e-08  3e-12
 6: -3.0387e+03 -9.5066e+03  6e+03  3e-08  3e-12
 7: -3.9761e+03 -1.0426e+04  6e+03  3e-08  4e-12
 8: -3.3866e+03 -9.8249e+03  6e+03  3e-08  3e-12
 9: -4.1507e+03 -8.5594e+03  4e+03  2e-08  3e-12
10: -4.2924e+03 -6.3860e+03  2e+03  6e-09  3e-12
11: -4.8749e+03 -6.9179e+03  2e+03  5e-09  3e-12
12: -5.0809e+03 -6.5351e+03  1e+03  4e-09  4e-12
13: -4.9462e+03 -6.4002e+03  1e+03  3e-09  4e-12
14: -4.9917e+03 -6.3668e+03  1e+03  3e-09  4e-12
15: -5.3897e+03 -5.7169e+03  3e+02  2e-12  4e-12
16: -5.4857e+03 -5.5120e+03  3e+01  4e-12  3e-12
17: -5.4966e+03 -5.4969e+03  3e-01  2e-12  4e-12
18: -5.4968e+03 -5.4968e+03  3e-03  1e-13  4e-12
Optimal solution foun